### Algorithm: NoisePop_PubSize
Under stratified random sampling, give a zCDP CI for population mean by adding noise to population mean. The sample sizes are considered public. 


In [ ]:
def make_map_partition_trans(trans: Transformation):
    return Transformation(
        ProductDomain(t.input_domain for t in trans),
        ProductDomain(t.output_domain for t in trans),
        function(|data: Vec<DI::Carrier>|{
            output = []
            for part, t in zip(data, trans):
                output.append(t(part))
            return output
            
        }),
        ProductMetric(trans[0].input_metric),
        ProductMetric(trans[0].output_metric),
        Stability_Map(|d_in: MI::Distance|{
            return max(t.map(d_in) for t in trans)
        })
    )     

In [ ]:
def make_lipschitz_mean(sample_sizes, strat_sizes):
    '''
    :param strat_sizes: the population size of each stratum
    :param sample_sizes: sample sizes in each stratum '''
    
    strat_weights = strat_sizes / np.sum(strat_sizes)
    return Transformation(
    ProductDomain<AllDomain<f64>>, 
    AllDomain<f64>,
    function(|samp_sums: Vec<f64>|{
        
        strat_means = samp_sums / sample_sizes
        return np.sum(stra_weights * strat_means) 
    }),
    ProductMetric<AbsoluteDistance<f64>>, 
    AbsoluteDistance<f64>,
    Stability_Map(|d_in: AbsoluteDistance<f64>|{
            sens = max(strat_weights / sample_sizes)
            return d_in * sens
        })
    )    

In [ ]:
def make_lipschitz_variance(sample_sizes, strat_sizes, scale_mean):
    '''
    :param strat_sizes: the population size of each stratum
    :param sample_sizes: sample sizes in each stratum 
    :param scale_mean: scale of gaussian noise added to mean'''
        
    strat_weights = strat_sizes / np.sum(strat_sizes)
    return Transformation(
    ProductDomain<AllDomain<f64>>, 
    AllDomain<f64>,
    function(|samp_sums:Vec<f64>|{
        
        strat_means = samp_sums / sample_sizes
        strat_var = (strat_sizes - sample_sizes) / strat_sizes * (strat_means * (1 - strat_means)) / (sample_sizes - 1)
        return np.sum(strat_weights * strat_means) + scale_mean ** 2
    }),
    ProductMetric<AbsoluteDistance<f64>>, 
    AbsoluteDistance<f64>,
    Stability_Map(|d_in: AbsoluteDistance<f64>|{
            sens = max(strat_weights ** 2 * (strat_sizes - sample_sizes) / strat_sizes / (sample_sizes - 1) / sample_sizes
            return d_in * sens
        })
    )    

In [ ]:
def make_noisestra_pubsize(sample_sizes, strat_sizes, scale_mean, scale_var):
    '''
    :param strat_sizes: the population size of each stratum
    :param sample_sizes: sample sizes in each stratum 
    :param scale_mean: scale of gaussian noise added to mean
    :param scale_var: scale of gaussian noise added to variance
    :return private estimators for mean and variance. '''
        
    trans_sums = make_map_partition_trans([
        make_sized_bound_sum(n,(0,1)) >> lipschitz_cast((0, n), TI = int, TO = float) 
        for n in sample_sizes])
    mean_meas = make_lipschitz_mean(sample_sizes, strat_sizes) >> make_base_gaussian(scale_mean),
    var_meas = make_lipschitz_variance(sample_sizes, strat_sizes, scale_mean) >> make_base_gaussian(scale_var)
    return  trans_sums >> composition([mean_meas, var_meas])